<a href="https://colab.research.google.com/github/heshameraqi/object-detection-dl-workshop/blob/master/Object_Detection_with_Deep_Learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Clone the workshop from Github & Install PyTorch-YOLOv3 Python requirements**



In [0]:
!git clone https://github.com/heshameraqi/object-detection-dl-workshop.git
%cd object-detection-dl-workshop/PyTorch-YOLOv3
!pip3 install -r requirements.txt

**Download YoloV3 weights**

In [0]:
%cd weights/
!bash download_yolov3_weights.sh
%cd ..

**Download MS-COCO dataset (just the metadata, because we won't re-train)**

In [0]:
%cd ../data/
!bash get_coco_dataset.sh
%cd ..

**Test the detector on sample images**

In [26]:
!python3 detect.py --image_folder data/samples/

Namespace(batch_size=1, checkpoint_model=None, class_path='data/coco.names', conf_thres=0.8, config_path='config/yolov3.cfg', image_folder='data/samples/', img_size=416, n_cpu=0, nms_thres=0.4, weights_path='weights/yolov3.weights')
Traceback (most recent call last):
  File "detect.py", line 65, in <module>
    classes = load_classes(opt.class_path)  # Extracts class labels from file
  File "/content/object-detection-dl-workshop/PyTorch-YOLOv3/utils/utils.py", line 22, in load_classes
    fp = open(path, "r")
FileNotFoundError: [Errno 2] No such file or directory: 'data/coco.names'
